Date	:10 November 2022

Team ID	:PNT2022TMID16515

Project Name :	AI-powered Nutrition Analyzer for Fitness Enthusiasts


# Data Collection

In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install keras
!pip install tensorflow

In [40]:
import tensorflow as tf
print(tf.version.VERSION)

2.7.2


# Image Preprocessing

In [3]:
from keras.preprocessing.image import ImageDataGenerator

# Image Data Augmentation

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

## Dataset 

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ATBg9o-x6XhRK4lUWYJS-cimkYYzzKC6C3LrPQCPnpuT',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-37xucknbbk5w2f'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
from io import BytesIO
import zipfile
unzip= zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [8]:
pwd

'/home/wsuser/work'

# Applying Image DataGenerator Functionality To Trainset And Testset

In [9]:
x_train = train_datagen.flow_from_directory(
    r'/home/wsuser/work/Dataset/TRAIN_SET/TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
x_test = test_datagen.flow_from_directory(
    r'/home/wsuser/work/Dataset/TEST_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 2626 images belonging to 5 classes.
Found 1055 images belonging to 5 classes.


In [10]:
#checking the number of classes
print(x_train.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [11]:
#checking the number of classes
print(x_test.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [12]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

# Model Building

1. Importing The Model Building Libraries

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout 

2. Initializing The Model

In [14]:
classifier = Sequential()

3. Adding CNN Layers

In [15]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())

4. Adding Dense Layers

In [16]:
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax'))

In [17]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)              

5. Configure The Learning Process

In [18]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

6. Train The Model

In [19]:
classifier.fit_generator(generator=x_train,steps_per_epoch = len(x_train),epochs=20, validation_data=x_test,validation_steps = len(x_test))

/tmp/wsuser/ipykernel_345/4293874847.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(generator=x_train,steps_per_epoch = len(x_train),epochs=20, validation_data=x_test,validation_steps = len(x_test))


Epoch 1/20
526/526 [==============================] - 28s 52ms/step - loss: 0.1370 - accuracy: 0.9520 - val_loss: 0.0079 - val_accuracy: 1.0000
Epoch 2/20
526/526 [==============================] - 26s 50ms/step - loss: 0.0290 - accuracy: 0.9909 - val_loss: 0.0138 - val_accuracy: 1.0000
Epoch 3/20
526/526 [==============================] - 27s 51ms/step - loss: 8.4672e-04 - accuracy: 1.0000 - val_loss: 0.0149 - val_accuracy: 1.0000
Epoch 4/20
526/526 [==============================] - 26s 50ms/step - loss: 0.1071 - accuracy: 0.9718 - val_loss: 0.0174 - val_accuracy: 0.9981
Epoch 5/20
526/526 [==============================] - 29s 54ms/step - loss: 7.0253e-04 - accuracy: 1.0000 - val_loss: 0.0741 - val_accuracy: 0.9678
Epoch 6/20
526/526 [==============================] - 26s 50ms/step - loss: 4.0716e-05 - accuracy: 1.0000 - val_loss: 0.0653 - val_accuracy: 0.9716
Epoch 7/20
526/526 [==============================] - 26s 50ms/step - loss: 5.7012e-05 - accuracy: 1.0000 - val_loss: 0.0187

In [21]:
classifier.history

7. Saving The Model

In [22]:
classifier.save('nutrition.h5')

In [24]:
!tar -zcvf Nutrition_new.tgz nutrition.h5

nutrition.h5


In [25]:
ls -1

Dataset/
nutrition.h5
Nutrition_new.tgz


8.IBM Watson Machine Learning

In [26]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 15.5 MB/s eta 0:00:01


In [27]:
from ibm_watson_machine_learning import APIClient
wml_credentials ={
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "L8gzXY3C6Vo5qAXVSs62XmIIz7JW7yno1-0h9aWmYynB"
}
client =APIClient(wml_credentials)

In [28]:
client =APIClient(wml_credentials)

In [29]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [35]:
space_uid = guid_from_space_name(client, 'Nutrition')
print("Space UID: "+ space_uid)

Space UID: 59a205dc-c259-483f-a631-0065bc6afdbc


In [36]:
client.set.default_space(space_uid)

'SUCCESS'

In [37]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [38]:
software_spec_uid= client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [42]:
model_details = client.repository.store_model(model='Nutrition_new.tgz',meta_props={
     client.repository.ModelMetaNames.NAME:"CNN",
     client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
     client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid}
                                             )                                          
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [43]:
model_id

'd3e4c7cd-542a-48f8-a71a-8e96091079ee'

In [44]:
client.repository.download(model_id,'nutrition_model.tar.gz')

Successfully saved model content to file: 'nutrition_model.tar.gz'


'/home/wsuser/work/nutrition_model.tar.gz'

8. Testing The Model

In [45]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("nutrition.h5")

In [52]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model = load_model("nutrition.h5")
img = image.load_img(r'/home/wsuser/work/Dataset/TRAIN_SET/TRAIN_SET/APPLES/r_70_100.jpg',grayscale=False,target_size= (64,64))
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
predict_x=model.predict(x) 
classes_x=np.argmax(predict_x,axis=-1)
classes_x

array([0])

In [53]:
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
result=str(index[classes_x[0]])
result

'APPLES'